In [64]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x16b434dd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x13f370250>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [65]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
model.invoke([HumanMessage(content="Hi, My name is Ibrahim and I am a Chief AI Engineer")])

AIMessage(content="Hello Ibrahim, it's a pleasure to meet you! \n\nThat's a fascinating title. As a Chief AI Engineer, you must be doing some incredible work. \n\nWhat kind of projects are you currently involved in? I'm always eager to learn more about the advancements being made in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 22, 'total_tokens': 91, 'completion_time': 0.125454545, 'prompt_time': 0.00133582, 'queue_time': 0.06942978, 'total_time': 0.126790365}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2e06aecb-8e5d-44bf-adbd-ed943d6cea8a-0', usage_metadata={'input_tokens': 22, 'output_tokens': 69, 'total_tokens': 91})

In [66]:
model.invoke(
    [
        HumanMessage(content="Hi, My name is Ibrahim and I am a Chief AI Engineer"),
        AIMessage(content="Hi Ibrahim, It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage("Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Ibrahim and that you are a Chief AI Engineer.  \n\nIs there anything else you'd like me to remember about you? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 97, 'total_tokens': 133, 'completion_time': 0.065454545, 'prompt_time': 0.00265865, 'queue_time': 0.069923968, 'total_time': 0.068113195}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e90222f-cf8a-45da-8757-d9f6fdaac49c-0', usage_metadata={'input_tokens': 97, 'output_tokens': 36, 'total_tokens': 133})

## Message History
We can use Message History class to wrap our model and make it stateful. This will keep track of inputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [67]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [68]:
config={"configurable": {"session_id": "chat1"}}

In [69]:
with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Ibrahim and I am a Chief AI Engineer")],
    config=config
)

AIMessage(content="Hello Ibrahim, it's nice to meet you! That's a fascinating role.  \n\nWhat kind of AI projects are you currently working on? I'm always interested in learning about the latest advancements in the field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 22, 'total_tokens': 71, 'completion_time': 0.089090909, 'prompt_time': 0.001324761, 'queue_time': 0.07109569, 'total_time': 0.09041567}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f618d5a-021e-409b-b8d8-3700abd879bc-0', usage_metadata={'input_tokens': 22, 'output_tokens': 49, 'total_tokens': 71})

In [70]:
## change the config --> session_id

config1 = {"configurable": {"session_id": "chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Ibrahim. 😊  \n\nI remembered from our first interaction.  How can I help you today?  \n'

In [71]:
store
config1 = {"configurable": {"session_id": "chat2"}}

In [72]:
## change the config --> session_id
response = with_message_history.invoke(
    [HumanMessage(content="Hey, my name is John")],
    config=config1
)
response.content

"Hello John, it's nice to meet you! 👋\n\nIs there anything I can help you with today?\n"

In [73]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  \n\nI remember that you told me earlier! 😊  How can I help you further?  \n'

## Prompt Templates
Prompt Teamplates help to turn raw user information into a format that the LLM can with with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


In [74]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Hey you are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [75]:
chain.invoke({
    "messages": [HumanMessage(content="Hi, my name is Ibrahim")]
})

AIMessage(content="Hi Ibrahim, it's nice to meet you! \n\nI'm happy to answer your questions to the best of my ability. What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 32, 'total_tokens': 74, 'completion_time': 0.076363636, 'prompt_time': 0.00147841, 'queue_time': 0.070192382, 'total_time': 0.077842046}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--95824a41-0289-4add-a717-5151bd7f5ea1-0', usage_metadata={'input_tokens': 32, 'output_tokens': 42, 'total_tokens': 74})

In [76]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [77]:
config = {"configurable": {"session_id": "chat_3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Ibrahim")],
    config=config
)

response.content

"Hello Ibrahim, it's nice to meet you!  \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n"

In [78]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Hey you are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke({
    "messages": [HumanMessage(content="Hi, my name is Ibrahim")], 
    "language": "Azerbaijani"
})

response.content

'Salam Ibrahim! 😊\n\nMən sənin köməkçi bir asistan olduğumuzu bildirdirdin. Mənə sorular sor və mənim gücümün bütünlüyünü istifadə edərək cavablandırmağa çalışacam.  \n\nNəyə maraqlanırsınız? \n\n'

### Let's do something more complex

In [80]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history, 
    input_messages_key="messages"
    )


In [81]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I a Ibrahim")], "language": "Azerbaijani"},
    config=config
)

response.content

'Salam, Ibrahim! 😊  \n\nMənim üçün xoşdur, sana kömək etmək.  Sənə bütün suallarına cavab verməyə çalışacam.  \n\nSənə nə sual etmək istəyirsiniz? 🤔\n\n'

### Managing the Conversation History

One important concept when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the cnotext window of the LLM. Therefore, it is imoiortant to add a step that limits the size of the messages you are passing in.

'trim_messages' helper function to reduce the amount of messages sent to the model. Allows us to specify how many tokens we are sending to the model. 

In [82]:
### Manage the conversation history

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45, 
    strategy="last", 
    token_counter=model, 
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice!"),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [83]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) 
    | prompt
    | model
)

response = chain.invoke({
    "messages": messages + [HumanMessage(content="What ice cream do i like?")],
    "language": "Azerbaijani",
}
)

response.content

"Mən bilmirəm, hansı dondurma sevdiyinizi. Məni bu barədə məlumatlandırın!  \n\n(I don't know what ice cream you like. Tell me about it!)\n"

In [84]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What math question did i ask for?")],
    "language": "English",
}
)

response.content

'You asked "What is 2 + 2?". 😊  \n\n\n\nIs there anything else I can help you with?\n'

In [85]:
## Let's wrap up this in the Message History

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat5"}}

In [86]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What's my name?")],
    "language": "English",
    }
)

response.content

"As a large language model, I have no memory of past conversations and do not know your name.\n\nIs there a name you'd like to use? 😊  \n\n"